In [ ]:
import uuid
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import when_matched, when_not_matched,col
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, FloatType

session = get_active_session()

In [ ]:
# CSV Schema
user_schema = StructType([StructField("USERNAME", StringType()), StructField("EMAIL", StringType()), StructField("IDENTIFIER", StringType())])
# Create a stage and upload the file to the stage
df = session.read.options({"field_delimiter": ";", "skip_header": 1}).schema(user_schema).csv("@sample_stage/username-or-email.csv")

csv_data = df.collect()

In [ ]:
formatted_datalist = []

for row in csv_data:
    formatted_datalist.append([row['USERNAME'], str(uuid.uuid4())+'@gmail.com'])
    
formatted_snowpark_df = session.create_dataframe(data=formatted_datalist, schema=["USERNAME", "EMAIL"])

target = session.table(["testing_db","public","users"])

target.merge(formatted_snowpark_df, (target["USERNAME"] == formatted_snowpark_df["USERNAME"]),
        [when_matched().update({"EMAIL": formatted_snowpark_df["EMAIL"]}), when_not_matched().insert(
            {"USERNAME": formatted_snowpark_df["USERNAME"]}
        )])

target.show()